In [1]:
# 데이터 불러오기

import pandas as pd
import numpy as np
import time






종목 차트정보 가져오는 중...
종목정보 가져오는 중...
지수정보 가져오는 중...
데이터를 불러왔습니다.


In [2]:
# 벡터연산 기반 시뮬레이션을 시뮬레이터와 투자방법으로 분리시킴


column_tradelog = ['code', 'name', 'section', 'idx_buydate', 'buydate','buyprice', 'idx_selldate', 'selldate', 'sellprice', 'period', 'rate','buy_index','sell_index','rate_index']
tradelog = pd.DataFrame(columns= column_tradelog)

n_test = 100000
np.random.seed(0)

start = time.time()


def Simulate(n_test, seed, ):
    print('투자 시뮬레이션을 시작합니다.')

    tradelog[['code','name', 'section', 'idx_firstdate']] = stockitems.loc[np.random.randint(0, len(stockitems), size=n_test), ['code', 'name', 'section','idx_firstdate']]

    tradelog['idx_buydate'] = tradelog.apply(lambda x: np.random.randint(x['idx_firstdate'], len(df_date)-1) ,axis = 1)
    tradelog['buydate'] = df_date.loc[tradelog['idx_buydate'],'date'].to_numpy()
    tradelog = pd.merge(tradelog, dailychart[['code','date','low','high']], left_on=['code','buydate'],  right_on=['code','date'], how='left')
    tradelog['buyprice'] = np.random.randint(tradelog['low'], tradelog['high']+1)
    tradelog = tradelog.drop(columns=['low', 'high'])

    tradelog['idx_selldate'] = tradelog['idx_buydate'] + np.random.randint(1, len(df_date) - tradelog['idx_buydate'])
    tradelog['selldate'] = df_date.loc[tradelog['idx_selldate'],'date'].to_numpy()
    tradelog = pd.merge(tradelog, dailychart[['code','date','low','high']], left_on=['code','selldate'],  right_on=['code','date'], how='left')
    tradelog['sellprice'] = np.random.randint(tradelog['low'], tradelog['high']+1)
    tradelog = tradelog.drop(columns=['low', 'high'])
    tradelog['period'] = tradelog['idx_selldate'] - tradelog['idx_buydate']
    tradelog['rate'] = (tradelog['sellprice'] - tradelog['buyprice']) / tradelog['buyprice'] * 100

    tradelog['buy_index'] = dailychart_KOSPI.loc[tradelog['idx_buydate'], 'close'].to_numpy()
    tradelog['sell_index'] = dailychart_KOSPI.loc[tradelog['idx_selldate'], 'close'].to_numpy()
    tradelog_index_KOSDAQ = tradelog.loc[tradelog['section'] == 'KOSDAQ'].index
    tradelog.loc[tradelog_index_KOSDAQ, 'buy_index'] = dailychart_KOSDAQ.loc[tradelog.loc[tradelog_index_KOSDAQ,'idx_buydate'], 'close'].to_numpy()
    tradelog.loc[tradelog_index_KOSDAQ, 'sell_index'] = dailychart_KOSDAQ.loc[tradelog.loc[tradelog_index_KOSDAQ,'idx_selldate'], 'close'].to_numpy()

    tradelog['rate_index'] =  (tradelog['sell_index'] - tradelog['buy_index']) / tradelog['buy_index'] * 100

    #불필요한 컬럼 제거
    tradelog = tradelog.drop(columns=['date_x','date_y', 'idx_firstdate'])

    
print('time : ', time.time() - start)
print('투자 시뮬레이션 데이터 로그를 저장합니다.')

tradelog.to_csv('tradelog3.csv')
print('시뮬레이션 데이터 로그를 저장하였습니다.')

    

투자 시뮬레이션을 시작합니다.
time :  2.798060178756714
투자 시뮬레이션 데이터 로그를 저장합니다.
시뮬레이션 데이터 로그를 저장하였습니다.


In [4]:
class Simulator:
    
    def __init__(self):
        column_tradelog = ['code', 'name', 'section', 'idx_buydate', 'buydate','buyprice', 'idx_selldate', 'selldate', 'sellprice', 'period', 'rate','buy_index','sell_index','rate_index']
        tradelog = pd.DataFrame(columns= column_tradelog)

    def initialize_chartdata(self, path_stockitems, path_chart, path_index): #종목 리스트, 종목차트 데이터, 지수차트 데이터 
        print('종목 차트정보 가져오는 중...')
        self.dailychart = pd.read_csv(path_chart)
        self.dailychart['date'] = pd.to_datetime(dailychart['date'], format='%Y%m%d')

        print("종목정보 가져오는 중...")
        stockitems_original = pd.read_csv('stockItems.csv')   #종목정보 가져오기
        stockitems_in_chart = dailychart['code'].unique()
        self.stockitems = stockitems_original.loc[stockitems_original['code'].isin(stockitems_in_chart)]

        # 날짜 데이터프레임 생성
        self.df_date = pd.DataFrame(dailychart['date'].unique(), columns=['date'])
        self.min_day = np.min(self.dailychart['date'])
        self.max_day = np.max(self.dailychart['date'])

        print("지수정보 가져오는 중...")
        dailychart_index = pd.read_csv(path_index)   #종목정보 가져오기

        dailychart_index['date'] = pd.to_datetime(dailychart_index['date'], format='%Y%m%d')
        dailychart_index = dailychart_index.loc[(min_day <= dailychart_index['date']) & ( dailychart_index['date'] <= max_day)].reset_index(drop=True)
        self.dailychart_KOSPI = dailychart_index.loc[dailychart_index['section'] == 'KOSPI'].reset_index(drop=True)
        self.dailychart_KOSDAQ = dailychart_index.loc[dailychart_index['section'] == 'KOSDAQ'].reset_index(drop=True)

        self.df_date['idx'] = self.df_date.index
        # 각 종목별 시작일에 날짜 인덱스를 추가
        tmp_df = pd.merge(dailychart.groupby('code', as_index=False)['date'].min(), df_date, on='date' )

        # 종목 데이터에 종목별 시작일과 시작날짜 인덱스를 조인
        self.stockitems = pd.merge(stockitems_original, tmp_df, on='code')
        self.stockitems = self.stockitems.rename(columns = {'date':'firstdate', 'idx':'idx_firstdate'})

        print('데이터를 불러왔습니다.')     
        
    def Evaluate():
        if dailychart

    def ExportTradeLog(self,path):
        tradelog.to_csv(path, encoding='utf-8', index=False)



SyntaxError: invalid syntax (<ipython-input-4-f617cf9f582f>, line 41)